In [35]:
import os 
workspace="/Users/tr/Documents/GitHub/Learning-Agents"
os.chdir(workspace)

In [36]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY', ''), 
    base_url="https://api.deepseek.com"
)

In [37]:
import json
json_path="flashcards/test_notes_20250606_132658_flashcards.json"
with open(json_path,"r") as f:
    data=json.load(f)

In [38]:
question=data[0].get('question')
context=data[0].get('context')

In [86]:
def format_deepseek_prompt():
    """生成用于DeepSeek的数学公式格式化提示"""
    return """
    在回答中，请严格遵循以下数学公式格式规范：
    
    1. 行内公式：
       - 使用单个美元符号：$formula$
       - 示例：$x$, $\\alpha$, $f(x)$
       - 变量、参数等单个符号也要使用数学模式：$x$ 而不是 x
    
    2. 行间公式：
       - 使用双美元符号：$$formula$$
       - 示例：$$\\min\\left(1, \\frac{\\pi(x')}{\\pi(x)}\\right)$$
       - 重要的多行公式或推导过程使用行间公式
    
    3. 数学符号规范：
       - 希腊字母：$\\alpha$, $\\beta$, $\\pi$ 等
       - 数学函数：$\\min$, $\\max$, $\\exp$, $\\log$ 等
       - 关系运算符：$\\leq$, $\\geq$, $\\neq$, $\\approx$ 等
       - 上下标使用花括号：$x_{t}$, $x^{2}$, $x_{i,j}$ 等
       - 分式：$\\frac{numerator}{denominator}$
    
    4. 格式化规则：
       - 所有数学符号和变量都必须在数学模式中
       - 公式前后要有适当的空格
       - 不要使用 Unicode 数学符号，使用 LaTeX 命令
       - 避免使用 \\[...\\] 或 \\(...\\) 格式
    """

prompt = f"""
        作为一个专业的数学和机器学习教育专家，请基于以下内容回答问题。
        请确保回答准确、清晰，并保持与原文一致的专业水平，不要求生成json等格式化数据，而是自然语言。
        
        根据数学格式化规则：
        {format_deepseek_prompt()}
        
        上下文内容：
        {context}

        问题：
        {question}
        """

In [87]:
response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {
                    "role": "system",
                    "content": "你是一个专业的数学和机器学习教育专家，精通将复杂的数学概念转换为清晰的FlashCards。请确保保留所有LaTeX公式。"
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=0.3,  # 降低温度以获得更确定性的输出
            max_tokens=4000,  # 增加最大token以处理长文本
            stream=False
        )

In [88]:
answer = response.choices[0].message.content.strip()

In [89]:
answer

'哈密顿蒙特卡洛（HMC）中的总能量守恒性质是其核心理论基础之一，源于哈密顿动力学的物理守恒定律。以下是关键要点：\n\n1. **哈密顿系统定义**  \n   在HMC中，系统状态由位置变量 $q$ 和动量变量 $p$ 描述，总能量（哈密顿量）定义为：\n   $$H(q,p) = U(q) + K(p)$$\n   其中：\n   - $U(q)$ 是势能，对应目标分布 $\\pi(q)$ 的负对数：$U(q) = -\\log \\pi(q)$\n   - $K(p)$ 是动能，通常取 $K(p) = \\frac{1}{2}p^T M^{-1}p$（$M$ 为质量矩阵）\n\n2. **守恒性质**  \n   在理想条件下（精确的哈密顿动力学模拟），系统满足：\n   $$\\frac{dH}{dt} = 0$$\n   即总能量 $H(q,p)$ 沿轨迹保持不变。这一性质来源于哈密顿方程：\n   $$\\frac{dq}{dt} = \\frac{\\partial H}{\\partial p}, \\quad \\frac{dp}{dt} = -\\frac{\\partial H}{\\partial q}$$\n\n3. **HMC中的意义**  \n   - 保证接受概率公式的合理性：接受率 $A = \\min\\left(1, \\exp(H_{\\text{current}} - H_{\\text{proposed}})\\right)$ 在理想情况下应为1\n   - 数值误差导致的实际应用中，能量会有微小波动，需通过离散化方法（如蛙跳法）和Metropolis校正来补偿\n\n4. **破坏守恒性的因素**  \n   - 离散化积分误差（步长 $\\epsilon$ 过大）\n   - 非对称动能函数（如使用Riemann流形HMC时）\n   - 约束条件处理不当（如边界反射）\n\n该性质使得HMC能高效探索状态空间，相比传统MCMC方法具有更低的样本相关性。'

In [79]:
import re 
def format_answer(answer):
    """格式化从DeepSeek获取的JSON格式答案"""
    # for json process
    answer_process = answer.strip()
    if answer_process.startswith('```json'):
        answer_process = answer_process[len("```json"):].strip()
    if answer_process.endswith('```'):
        answer_process = answer_process[:-len("```")].strip()
    answer_process = answer_process.lstrip('\ufeff')
    # 尝试解析JSON
    answer_data = json.loads(answer_process)
    # 构建格式化文本
    formatted_parts = []
    
    # 添加主要描述
    description = answer_data.get('answer', {}).get('description', '')
    if description:
        formatted_parts.append(description.strip())
    # 添加详细信息
    details = answer_data.get('answer', {}).get('details', [])
    if details:
        for detail in details:
            term = detail.get('term', '').strip()
            explanation = detail.get('explanation', '').strip()
            if term and explanation:
                # 添加空行和详细信息
                formatted_parts.extend(['', f"**{term}**：{explanation}"])
    # 添加附注
    note = answer_data.get('answer', {}).get('note', '')
    if note:
        # 添加空行和附注
        formatted_parts.extend(['', f"**附注**：{note.strip()}"])
    
    text = '\n'.join(formatted_parts)
    
    # 处理数学公式和格式化
    def clean_math_text(text):
        # 统一处理Unicode数学符号
        unicode_math = {
            '𝑥': 'x', '𝑟': 'r', '𝑞': 'q', '𝑝': 'p',
            '𝜋': '\\pi', '𝜃': '\\theta', '𝜇': '\\mu',
            '′': "'", '⋅': '\\cdot', '∼': '\\sim',
            '∆': '\\Delta', 'α': '\\alpha', 'β': '\\beta'
        }
        for unicode_char, latex_char in unicode_math.items():
            text = text.replace(unicode_char, latex_char)
        
        # 处理数学公式
        text = re.sub(r'\\Delta\s+H', '\\Delta H', text)  # 修复Delta H的间距
        text = re.sub(r'\$\((.*?)\)\$', r'$\1$', text)  # 移除公式中多余的括号
        text = re.sub(r'\(\s*x\s*,\s*r\s*\)', '(x,r)', text)  # 标准化坐标对
        text = re.sub(r'\$\s*,\s*\$', ', ', text)  # 修复被错误分割的公式
        
        # 处理绝对值符号
        text = text.replace('|', '\\|')  # 将普通竖线替换为LaTeX绝对值符号
        
        # 确保公式中的空格正确
        text = re.sub(r'(?<=\w)(?=[\\$])', ' ', text)  # 在公式前添加空格
        text = re.sub(r'(?<=[\\$])(?=\w)', ' ', text)  # 在公式后添加空格
        
        # 处理多行公式
        text = re.sub(r'\n\s*(?=[\\$])', ' ', text)  # 合并跨行的公式
        text = re.sub(r'(?<=[\\$])\s*\n', ' ', text)  # 合并跨行的公式
        
        # 处理段落格式
        text = re.sub(r'\n{3,}', '\n\n', text)  # 减少多余的空行
        text = re.sub(r'(?<=。)\s*\n\s*(?=\S)', '\n\n', text)  # 在句子之间添加适当的空行
        
        return text.strip()
    
    # 应用清理和格式化
    # formatted_text = clean_math_text(text)
    return answer_data
        

In [81]:
answer_data=format_answer(answer)
answer_data

{'answer': {'description': '哈密顿蒙特卡洛(HMC)中的总能量守恒性质是指在模拟哈密顿动力学过程中，系统的总能量（即哈密顿量）在理想情况下保持不变。这一性质源于哈密顿动力学的可逆性和辛结构，确保了采样过程在相空间中的体积守恒（Liouville定理）。数学上，哈密顿量定义为 $H(q,p) = U(q) + K(p)$，其中 $U(q)$ 是势能（通常对应目标分布的负对数概率），$K(p)$ 是动能（通常为 $p^T M^{-1} p/2$）。在数值模拟中，由于离散化误差，总能量会轻微波动，但通过使用辛积分器（如蛙跳法）可最小化这种误差。',
  'details': [{'term': '哈密顿量 (Hamiltonian)',
    'explanation': '系统的总能量，$H(q,p) = U(q) + K(p)$，其中 $q$ 是位置变量，$p$ 是动量变量。'},
   {'term': '辛积分器 (Symplectic Integrator)',
    'explanation': '一种保持哈密顿系统几何结构的数值积分方法（如蛙跳法），能长期近似保持总能量。'},
   {'term': 'Liouville定理',
    'explanation': '哈密顿动力学在相空间中保持体积不变，是HMC满足细致平衡条件的基础。'}],
  'note': '实际HMC中，离散化会导致能量不严格守恒，需通过Metropolis-Hastings步骤校正以保持理论收敛性。'}}

In [66]:
for keys in answer_data.get('answer').keys:
    print()

'在哈密顿蒙特卡洛(Hamiltonian Monte Carlo, HMC)中，总能量守恒性质是指系统在理想情况下（即数值积分误差为零时）的哈密顿量$H(\\mathbf{q}, \\mathbf{p})$保持恒定。'